In [1]:
#Importing necessary libraries
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import json
import numpy as np
import pandas as pd
from datasets import load_dataset
import torch
from torch import nn
from scipy.stats import pearsonr
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split

Maybe the 50 dimensions is a little too small and I should start with 128

In [2]:
# Load your fine-tuned local model 
model_path = '/home/mendu/Thesis/data/musiccaps/new_embedding_model'
model = SentenceTransformer(model_path)

In [3]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [4]:
# captions = pd.read_csv('/home/mendu/Thesis/data/musiccaps/lp-music-caps_caption_writing.csv', index_col = [0])

In [5]:
msd_dataset = load_dataset('seungheondoh/LP-MusicCaps-MSD')

In [6]:
train = pd.DataFrame(msd_dataset['train'])
# test = pd.DataFrame(msd_dataset['test'])
# valid = pd.DataFrame(msd_dataset['valid'])

In [7]:
sentences = train['caption_writing'].to_list()

## Training a PCA

In [8]:
# Generate embeddings for your sentences using the fine-tuned model
embedded_sentences = model.encode(sentences, show_progress_bar=True, convert_to_numpy=True)

# Number of PCA components (e.g., reduce to 50 dimensions)
num_components = 50
# obj = PCA(n_components=num_components)

# Fit the PCA model to the embedded sentences (this will find the principal components)
# pca_embeddings = obj.fit_transform(embedded_sentences)

Batches:   0%|          | 0/13903 [00:00<?, ?it/s]

In [9]:
# # Project the PCA embeddings back to the original space
# projected_embeddings = obj.inverse_transform(pca_embeddings)

# # Initialize an empty list to store the Pearson correlation coefficients
# pearsons_correlations = []

# # Calculate Pearson's correlation for each pair of original and projected embeddings
# for original, projected in zip(embedded_sentences, projected_embeddings):
#     # Compute Pearson's r
#     corr, _ = pearsonr(original, projected)
#     pearsons_correlations.append(corr)

# # If you want to compute a single Pearson's correlation coefficient for all data
# # Concatenate all embeddings and compute the correlation
# flat_original = embedded_sentences.flatten()
# flat_projected = projected_embeddings.flatten()
# overall_corr, _ = pearsonr(flat_original, flat_projected)

# # print("Pearsons correlation for each embedding pair:", pearsons_correlations)
# print("Overall Pearson's correlation:", overall_corr)

In [10]:
# print("Original embeddings shape:", embedded_sentences.shape)
# print("PCA-reduced embeddings shape:", pca_embeddings.shape)

## Training an autoencoder

In [11]:
# import torch
# from torch import nn

# class Autoencoder(nn.Module):
#     def __init__(self, input_size, encoding_size):
#         super(Autoencoder, self).__init__()
#         # Encoder
#         self.encoder = nn.Sequential(
#             nn.Linear(input_size, encoding_size),
#             nn.ReLU(True)
#         )
#         # Decoder
#         self.decoder = nn.Sequential(
#             nn.Linear(encoding_size, input_size),
#             nn.Sigmoid()  # Using Sigmoid because embeddings are likely normalized
#         )

#     def forward(self, x):
#         encoded = self.encoder(x)
#         decoded = self.decoder(encoded)
#         return decoded
    

class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_size):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(True),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.BatchNorm1d(256),  # Added batch normalization
            nn.Linear(256, 128),  # Added another layer
            nn.ReLU(True),
            nn.Linear(128, encoding_size),  # Adjusted the size of the encoding layer
            nn.ReLU(True)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(encoding_size, 128),  # Adjusted the size of the decoding layer
            nn.ReLU(True),
            nn.Linear(128, 256),  # Added another layer
            nn.ReLU(True),
            nn.BatchNorm1d(256),  # Added batch normalization
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, input_size),
            nn.Sigmoid()  # Using Sigmoid because embeddings are likely normalized
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [12]:
embedded_sentences.shape

(444865, 768)

In [13]:
# Convert the NumPy array of embeddings to a PyTorch tensor
embedded_sentences_tensor = torch.tensor(embedded_sentences, dtype=torch.float32)

# Create a dataset and a dataloader
dataset = TensorDataset(embedded_sentences_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)

# # Split dataset into training and validation sets
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_set, val_set = random_split(dataset, [train_size, val_size])

# train_loader = DataLoader(train_set, batch_size=64, shuffle=True, drop_last=True)
# val_loader = DataLoader(val_set, batch_size=64, shuffle=False)

In [14]:
# # Convert the NumPy array of embeddings to a PyTorch tensor
# embedded_sentences_tensor = torch.tensor(embedded_sentences, dtype=torch.float32)

# # Create a dataset and a dataloader without labels
# dataset = TensorDataset(embedded_sentences_tensor)

# # Split dataset into training and validation sets
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_set, val_set = random_split(dataset, [train_size, val_size])

# # Create dataloaders without labels
# train_loader = DataLoader(train_set, batch_size=64, shuffle=True, drop_last=True)
# val_loader = DataLoader(val_set, batch_size=64, shuffle=False)


In [15]:
# print(len(dataset))  # Number of samples in the dataset
# print(dataset[0][0].shape)  # Shape of the first sample in the dataset
# print(dataset[0][0].ndim)
# print(embedded_sentences_tensor.shape)


In [16]:
# # Iterate over the batches
# for batch_idx, (data, target) in enumerate(train_loader):
#     # Print the shape of the data in the first batch
#     print(f"Batch {batch_idx}: Data shape: {data.shape}")


In [17]:
# Instantiate the autoencoder
input_size = embedded_sentences.shape[1]
encoding_size = 64  # change this to whatever size you want to encode down to
autoencoder = Autoencoder(input_size=input_size, encoding_size=encoding_size)

# Define loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.L1Loss()  # MAE
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)

In [18]:
input_size

768

In [19]:
epochs = 20  # Set this to the number of epochs to train for
for epoch in range(epochs):
    for data in dataloader:
        inputs = data[0]
        # Forward pass
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/20, Loss: 0.3479156792163849
Epoch 2/20, Loss: 0.3374408185482025
Epoch 3/20, Loss: 0.33837732672691345
Epoch 4/20, Loss: 0.337697833776474
Epoch 5/20, Loss: 0.337202787399292
Epoch 6/20, Loss: 0.33835944533348083
Epoch 7/20, Loss: 0.33562734723091125
Epoch 8/20, Loss: 0.3408569097518921
Epoch 9/20, Loss: 0.33289599418640137
Epoch 10/20, Loss: 0.332951158285141
Epoch 11/20, Loss: 0.3391749858856201
Epoch 12/20, Loss: 0.3392314910888672
Epoch 13/20, Loss: 0.33417224884033203
Epoch 14/20, Loss: 0.342439740896225
Epoch 15/20, Loss: 0.3379240036010742
Epoch 16/20, Loss: 0.33648166060447693
Epoch 17/20, Loss: 0.33651337027549744
Epoch 18/20, Loss: 0.3340362310409546
Epoch 19/20, Loss: 0.3365885317325592
Epoch 20/20, Loss: 0.3371446132659912


In [20]:
# # Define the EarlyStopping class
# class EarlyStopping:
#     def __init__(self, patience=5, verbose=False):
#         self.patience = patience
#         self.verbose = verbose
#         self.counter = 0
#         self.best_loss = float('inf')
#         self.early_stop = False

#     def __call__(self, val_loss, model):
#         if val_loss < self.best_loss:
#             self.best_loss = val_loss
#             self.counter = 0
#         else:
#             self.counter += 1
#             if self.counter >= self.patience:
#                 self.early_stop = True
#                 if self.verbose:
#                     print("Early stopping")



In [21]:
# # Instantiate the EarlyStopping object
# early_stopping = EarlyStopping(patience=5, verbose=True)

# epochs = 50
# # Training loop with early stopping
# for epoch in range(epochs):
#     # Train the model
#     model.train()  # Set the model to training mode
#     for batch_idx, data in enumerate(train_loader):
#         optimizer.zero_grad()
#         # Unpack the batch
#         embeddings = data[0]  # Assuming the embeddings are in the first element of the tuple
#         output = model(embeddings)
#         loss = criterion(output, embeddings)  # Use embeddings as target since it's an autoencoder
#         loss.backward()
#         optimizer.step()
        
#     # Validate the model
#     model.eval()  # Set the model to evaluation mode
#     val_loss = 0.0
#     with torch.no_grad():
#         for data in val_loader:
#             embeddings = data[0]  # Assuming the embeddings are in the first element of the tuple
#             output = model(embeddings)
#             val_loss += criterion(output, embeddings).item()

#     val_loss /= len(val_loader)

#     print(f'Epoch {epoch+1}/{epochs}, Validation Loss: {val_loss}')

#     # Call early stopping
#     early_stopping(val_loss, model)

#     if early_stopping.early_stop:
#         print("Early stopping")
#         break

- This constant loss means that we should try going smaller in the embedding space, go to 64 or 32. 
- DO early .
- Higher learning rate

In [22]:
# Switch the autoencoder to evaluation mode
autoencoder.eval()

# Process the entire dataset to obtain the decoded (projected) embeddings
encoded_embeddings = autoencoder.encoder(embedded_sentences_tensor).detach().numpy()
decoded_embeddings = autoencoder.decoder(torch.from_numpy(encoded_embeddings)).detach().numpy()

# Calculate Pearson's correlation
from scipy.stats import pearsonr
correlations = np.array([pearsonr(original, decoded)[0] for original, decoded in zip(embedded_sentences, decoded_embeddings)])

# print('Pearson correlation for each embedding pair:', correlations)
print('Mean Pearson correlation:', np.mean(correlations))

Mean Pearson correlation: 0.7092163535337833


In [23]:
np.save('/home/mendu/Thesis/data/musiccaps/auto_encoder/encoded_embeddings.npy', encoded_embeddings)

Change the metric, reconstruction loss

In [24]:
print("Original embeddings shape:", embedded_sentences_tensor.shape)

# Switch autoencoder to evaluation mode
autoencoder.eval()

# Process the entire dataset to obtain the encoded embeddings
encoded_embeddings = autoencoder.encoder(embedded_sentences_tensor).detach()

print("Entire dataset encoded embeddings shape:", encoded_embeddings.shape)

Original embeddings shape: torch.Size([444865, 768])
Entire dataset encoded embeddings shape: torch.Size([444865, 64])


## Trying to get the word embeddings of the 8 class labels

In [28]:
import os

os.listdir('/home/mendu/Thesis/data/musiccaps/new_embedding_model')

['tokenizer_config.json',
 'model.safetensors',
 'tokenizer.json',
 'vocab.json',
 '1_Pooling',
 'merges.txt',
 'sentence_bert_config.json',
 'config_sentence_transformers.json',
 'modules.json',
 'special_tokens_map.json',
 'README.md',
 'config.json']

In [29]:
import numpy as np
from transformers import AutoTokenizer

# Load the fine-tuned tokenizer
tokenizer = AutoTokenizer.from_pretrained('/home/mendu/Thesis/data/musiccaps/new_embedding_model/')  # Update the path accordingly

# Load the 64-dimensional word embeddings
embeddings_reduced = np.load('/home/mendu/Thesis/data/musiccaps/auto_encoder/encoded_embeddings.npy')  # Update with the correct .npy file path

# Function to get the embedding of a specific word
def get_word_embedding(word):
    # Tokenize the word to get its ID
    token_id = tokenizer.encode(word, add_special_tokens=False)
    if not token_id:
        raise ValueError(f"The word '{word}' was not found in the tokenizer's vocabulary.")
    elif len(token_id) > 1:
        raise ValueError(f"The input text '{word}' corresponds to multiple tokens.")
    token_id = token_id[0]  # We only expect one token ID for a single word input
    
    # Fetch the corresponding embedding
    word_embedding = embeddings_reduced[token_id]
    return word_embedding

# Example usage:
word = "rock"  # Replace with the word you're interested in
embedding_of_music = get_word_embedding(word)
print(f"The embedding for the word '{word}' is:", embedding_of_music)

The embedding for the word 'music' is: [  0.       234.11252   55.04939   83.73867  119.17586   24.50513
  79.37317    0.         0.       107.77891   85.39018   32.598988
 116.71515    0.         0.       177.07715   55.900036  50.565147
  70.38419  109.61758    0.       103.22257  105.240326  67.54228
  79.07263    0.        78.31037    0.        46.628445 139.69655
  41.480885 139.17657  179.46729  204.76526  144.35342  128.00192
  36.790775  74.43881    0.       107.29045   92.97456   16.40067
  58.436283   0.       112.11439   86.0272     0.         0.
  75.80277   79.70243    0.         0.        76.836876  98.40882
  91.30781   37.967175  95.21652    0.        40.375088   0.
  74.799095 110.38203    0.         0.      ]


In [30]:
# Example usage:
embedding_of_rock = get_word_embedding("rock")
print(f"The embedding for the word rock is:", embedding_of_rock)

The embedding for the word rock is: [  0.        126.707634   56.95655    30.001554   90.95441    67.627846
 127.68996     0.          0.         59.838825  157.24373    65.84189
  49.76614     0.          0.        152.53204    44.524162  109.71577
  75.87621   170.42998     0.         51.2337    148.76892   102.406395
 132.56421     0.         56.189133    0.         61.006737  139.45685
  88.97968    88.43195    81.91134   189.14525   267.908      75.82866
  22.694725   74.57384     0.        153.54211    56.44078     2.5905855
  98.41678     0.         98.76408    60.717625    0.          0.
  65.88503    38.88233     0.          0.         89.83166    64.59985
 144.08324    42.727947  107.228676    0.         61.92044     0.
 125.440765   74.22465     0.          0.       ]


In [39]:
get_word_embedding("new age")

ValueError: The input text 'new age' corresponds to multiple tokens.

In [ ]:
# Load your fine-tuned local model 
model_path = '/home/mendu/Thesis/data/musiccaps/new_embedding_model'
model = SentenceTransformer(model_path)

# Load the tokenizer corresponding to the pre-trained model
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')

# Access the vocabulary of the tokenizer
vocab = tokenizer.get_vocab()

# Assuming you already have the fine-tuned embeddings
# For demonstration purposes, let's generate random embeddings
fine_tuned_embeddings = np.random.rand(len(vocab), model.get_sentence_embedding_dimension())

# Create metadata mapping embeddings to words
metadata = {word: embedding.tolist() for word, embedding in zip(vocab.keys(), fine_tuned_embeddings)}

# Save the embeddings and metadata
np.save('/home/mendu/Thesis/data/musiccaps/auto_encoder/fine_tuned_embeddings.npy', fine_tuned_embeddings)
with open('metadata.json', 'w') as f:
    json.dump(metadata, f)


In [ ]:
encoded_embeddings = np.load('/home/mendu/Thesis/data/musiccaps/auto_encoder/encoded_embeddings.npy')

In [ ]:
encoded_embeddings

In [25]:
'''
epochs = 50  # Set this to the number of epochs to train for
for epoch in range(epochs):
    for data in dataloader:
        inputs = data[0]
        # Forward pass
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')
'''

"\nepochs = 50  # Set this to the number of epochs to train for\nfor epoch in range(epochs):\n    for data in dataloader:\n        inputs = data[0]\n        # Forward pass\n        outputs = autoencoder(inputs)\n        loss = criterion(outputs, inputs)\n\n        # Backward pass\n        optimizer.zero_grad()\n        loss.backward()\n        optimizer.step()\n        \n    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')\n"